In [1]:
# import kagglehub

# # Download latest version
# path = kagglehub.dataset_download("shusrith/machine-trainslation")

# print("Path to dataset files:", path)

In [2]:
import pandas as pd
from sklearn.utils import shuffle

df = pd.read_csv(f"EnglishOrSpanish/output_joint1.csv")
df = shuffle(df)
df

,English,Spanish
106178,"[2, 290, 2762, 1666, 369, 398, 778, 708, 265, ...","[2, 290, 302, 311, 1614, 265, 369, 986, 500, 2..."
23834,"[2, 300, 3634, 449, 332, 1436, 224, 3, 0, 0, 0...","[2, 434, 1474, 918, 322, 321, 1800, 224, 3, 0,..."
111637,"[2, 657, 304, 1820, 4414, 328, 426, 404, 334, ...","[2, 375, 22651, 4592, 294, 444, 336, 4106, 690..."
102138,"[2, 805, 459, 304, 526, 382, 398, 871, 1251, 1...","[2, 879, 2687, 265, 2135, 321, 314, 1331, 9191..."
127930,"[2, 300, 778, 304, 671, 300, 778, 304, 575, 70...","[2, 2798, 500, 9411, 500, 12706, 292, 500, 106..."
...,...,...
84833,"[2, 290, 1401, 462, 7296, 1235, 224, 3, 0, 0, ...","[2, 1206, 290, 2763, 352, 7644, 224, 3, 0, 0, ..."
27872,"[2, 300, 5408, 334, 4557, 224, 3, 0, 0, 0, 0, ...","[2, 331, 5849, 393, 8476, 224, 3, 0, 0, 0, 0, ..."
12382,"[2, 555, 2701, 4037, 224, 3, 0, 0, 0, 0, 0, 0,...","[2, 3403, 393, 5508, 224, 3, 0, 0, 0, 0, 0, 0,..."
84177,"[2, 300, 1366, 272, 1055, 1024, 384, 3688, 224...","[2, 321, 1429, 571, 2728, 294, 290, 621, 5655,..."


In [3]:
import ast

df["English"] = df["English"].apply(ast.literal_eval)
df["Spanish"] = df["Spanish"].apply(ast.literal_eval)
df

,English,Spanish
106178,"[2, 290, 2762, 1666, 369, 398, 778, 708, 265, ...","[2, 290, 302, 311, 1614, 265, 369, 986, 500, 2..."
23834,"[2, 300, 3634, 449, 332, 1436, 224, 3, 0, 0, 0...","[2, 434, 1474, 918, 322, 321, 1800, 224, 3, 0,..."
111637,"[2, 657, 304, 1820, 4414, 328, 426, 404, 334, ...","[2, 375, 22651, 4592, 294, 444, 336, 4106, 690..."
102138,"[2, 805, 459, 304, 526, 382, 398, 871, 1251, 1...","[2, 879, 2687, 265, 2135, 321, 314, 1331, 9191..."
127930,"[2, 300, 778, 304, 671, 300, 778, 304, 575, 70...","[2, 2798, 500, 9411, 500, 12706, 292, 500, 106..."
...,...,...
84833,"[2, 290, 1401, 462, 7296, 1235, 224, 3, 0, 0, ...","[2, 1206, 290, 2763, 352, 7644, 224, 3, 0, 0, ..."
27872,"[2, 300, 5408, 334, 4557, 224, 3, 0, 0, 0, 0, ...","[2, 331, 5849, 393, 8476, 224, 3, 0, 0, 0, 0, ..."
12382,"[2, 555, 2701, 4037, 224, 3, 0, 0, 0, 0, 0, 0,...","[2, 3403, 393, 5508, 224, 3, 0, 0, 0, 0, 0, 0,..."
84177,"[2, 300, 1366, 272, 1055, 1024, 384, 3688, 224...","[2, 321, 1429, 571, 2728, 294, 290, 621, 5655,..."


In [4]:
import torch
from torch.utils.data import DataLoader, TensorDataset

train = TensorDataset(
    torch.tensor(df["English"][:100000].tolist(), dtype=torch.long),
    torch.tensor(df["Spanish"][:100000].tolist(), dtype=torch.long),
)
test = TensorDataset(
    torch.tensor(df["English"][100000:].tolist(), dtype=torch.long),
    torch.tensor(df["Spanish"][100000:].tolist(), dtype=torch.long),
)
train_loader = DataLoader(train, batch_size=256, shuffle=True, drop_last=True)
test_loader = DataLoader(test, batch_size=256, shuffle=False, drop_last=True)

In [5]:
import json

with open(f"EnglishOrSpanish/vocab.json", "r") as f:
    vocab = json.load(f)

In [6]:
import torch
import torch.nn as nn


class Encoder(nn.Module):
    def __init__(self, vocab_size, hidden_size, num_layers=4, dropout=0.3):
        super(Encoder, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.vocab_size = vocab_size

        self.embedding = nn.Embedding(vocab_size, hidden_size, padding_idx=14453)
        self.lstm = nn.LSTM(
            hidden_size,
            hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout,
        )

        self.layer_norm = nn.LayerNorm(hidden_size)
        self.hidden_norm = nn.LayerNorm(hidden_size)
        self.cell_norm = nn.LayerNorm(hidden_size)
        self.dropout = nn.Dropout(dropout)

        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.LSTM):
            for name, param in module.named_parameters():
                if "weight" in name:
                    nn.init.xavier_uniform_(param)
                elif "bias" in name:
                    nn.init.zeros_(param)

    def forward(self, x):
        embedded = self.dropout(self.embedding(x))
        output, (hidden, cell) = self.lstm(embedded)
        output = self.layer_norm(output)
        hidden = self.hidden_norm(hidden)
        cell = self.cell_norm(cell)
        return output, hidden, cell

In [7]:
class Attention(nn.Module):
    def __init__(self, hidden_size):
        super(Attention, self).__init__()
        self.hidden_size = hidden_size

        self.attn = nn.Linear(hidden_size, hidden_size)
        self.v = nn.Linear(hidden_size, 1, bias=False)

    def forward(self, hidden, encoder_output):
        hidden = hidden.unsqueeze(1)
        scores = self.attn(encoder_output)
        scores = torch.tanh(scores + hidden)
        attention = self.v(scores).squeeze(2)
        return nn.functional.softmax(attention, dim=1)

In [8]:
import torch
import torch.nn as nn


class Decoder(nn.Module):
    def __init__(self, vocab_size, hidden_size, attention, num_layers=4, dropout=0.3):
        super(Decoder, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.vocab_size = vocab_size

        self.embedding = nn.Embedding(vocab_size, hidden_size, padding_idx=23722)

        self.lstm = nn.LSTM(
            2 * hidden_size,
            hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout,
        )
        self.attention = attention

        self.layer_norm = nn.LayerNorm(hidden_size)
        self.hidden_norm = nn.LayerNorm(hidden_size)
        self.cell_norm = nn.LayerNorm(hidden_size)

        self.fc = nn.Linear(hidden_size, vocab_size)
        self.dropout = nn.Dropout(dropout)
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.LSTM):
            for name, param in module.named_parameters():
                if "weight" in name:
                    nn.init.xavier_uniform_(param)
                elif "bias" in name:
                    nn.init.zeros_(param)

    def forward(self, x, hidden, cell, encoder_output):
        attention_scores = self.attention(hidden[-1], encoder_output)
        context = torch.bmm(attention_scores.unsqueeze(1), encoder_output)
        embedded = self.dropout(self.embedding(x)).unsqueeze(1)
        embedded = torch.cat([embedded, context], dim=2)
        output, (hidden, cell) = self.lstm(embedded, (hidden, cell))
        output = self.layer_norm(output)
        hidden = self.hidden_norm(hidden)
        cell = self.cell_norm(cell)
        output = self.fc(output)
        return output, hidden, cell

In [9]:
class Seq2Seq:
    def __init__(self, encoder, decoder, device, teacher_forcing_ratio):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        self.vocab_size = self.decoder.vocab_size
        self.teacher_forcing_ratio = teacher_forcing_ratio

    def train(
        self,
        train_loader,
        test_loader,
        enc_optimizer,
        dec_optimizer,
        criterion,
        device,
        encoder_scheduler,
        decoder_scheduler,
        num_epochs,
    ):
        for epoch in range(num_epochs):
            print(f"\nEpoch {epoch+1}/{num_epochs}")
            self.encoder.train()
            self.decoder.train()
            epoch_loss = 0
            epoch_acc = 0
            progress_bar = tqdm(train_loader, desc="Training", leave=False)

            for src, trg in progress_bar:
                src, trg = src.to(device), trg.to(device)

                batch_size, trg_len = trg.shape

                enc_optimizer.zero_grad()
                dec_optimizer.zero_grad()

                encoder_out, hidden, cell = self.encoder(src)
                outputs = torch.zeros(batch_size, trg_len, self.vocab_size).to(
                    self.device
                )
                x = trg[:, 0]
                for t in range(1, trg_len):
                    output, hidden, cell = self.decoder(x, hidden, cell, encoder_out)
                    outputs[:, t, :] = output.squeeze(1)
                    teacher_force = torch.rand(1).item() < self.teacher_forcing_ratio
                    x = trg[:, t] if teacher_force else output.argmax(dim=1)
                loss = criterion(
                    outputs[:, 1:].reshape(-1, self.vocab_size), trg[:, 1:].reshape(-1)
                )
                loss.backward()
                enc_optimizer.step()
                dec_optimizer.step()

                epoch_loss += loss.item()

                preds = outputs.argmax(dim=2)
                correct = (preds == trg).float().sum().item()
                total = trg.numel()
                batch_acc = correct / total
                epoch_acc += batch_acc

                progress_bar.set_postfix(
                    loss=f"{loss.item():.4f}", acc=f"{batch_acc:.4f}"
                )

            train_loss = epoch_loss / len(train_loader)
            train_acc = epoch_acc / len(train_loader)

            # ---------------- VALIDATION ---------------- #
            self.encoder.eval()
            self.decoder.eval()
            val_epoch_loss = 0
            val_epoch_acc = 0
            progress_bar = tqdm(test_loader, desc="Validating", leave=False)

            with torch.no_grad():
                for src, trg in progress_bar:
                    src, trg = src.to(device), trg.to(device)
                    batch_size, trg_len = trg.shape
                    encoder_out, hidden, cell = self.encoder(src)
                    outputs = torch.zeros(batch_size, trg_len, self.vocab_size).to(
                        self.device
                    )
                    x = trg[:, 0]
                    for t in range(1, trg_len):
                        output, hidden, cell = self.decoder(x, hidden, cell, encoder_out)
                        outputs[:, t, :] = output
                        x = output.argmax(dim=1)

                    loss = criterion(
                        outputs[:, 1:].reshape(-1, self.vocab_size),
                        trg[:, 1:].reshape(-1),
                    )
                    val_epoch_loss += loss.item()

                    preds = outputs.argmax(dim=2)
                    correct = (preds == trg).float().sum().item()
                    total = trg.numel()
                    batch_acc = correct / total
                    val_epoch_acc += batch_acc

                    progress_bar.set_postfix(
                        loss=f"{loss.item():.4f}", acc=f"{batch_acc:.4f}"
                    )

            val_loss = val_epoch_loss / len(test_loader)
            val_acc = val_epoch_acc / len(test_loader)
            print(
                f"Train loss : {train_loss}, Train accuracy : {train_acc}, Val_loss : {val_loss}, val accuracy : {val_acc}"
            )
            encoder_scheduler.step(val_loss)
            decoder_scheduler.step(val_loss)
        return train_loss, train_acc, val_loss, val_acc

    def predict(self, src, trg):
        self.encoder.eval()
        self.decoder.eval()
        with torch.no_grad():
            src, trg = src.to(self.device), trg.to(self.device)
            batch_size, trg_len = trg.shape
            _, hidden, cell = self.encoder(src)
            outputs = torch.zeros(batch_size, trg_len, self.vocab_size).to(self.device)
            x = trg[:, 0]
            for t in range(1, trg_len):
                output, hidden, cell = self.decoder(x, hidden, cell)
                outputs[:, t, :] = output
                x = output.argmax(dim=1)
            return outputs

In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

vocab_size = len(vocab)

hidden_size = 128
encoder = Encoder(vocab_size, hidden_size).to(device)
attention = Attention(hidden_size)
decoder = Decoder(vocab_size, hidden_size, attention).to(device)
seq2seq = Seq2Seq(encoder, decoder, device, 0.5)
criterion = nn.CrossEntropyLoss(ignore_index=23722)
encoder_optimizer = optim.Adam(encoder.parameters(), lr=0.0005, weight_decay=1e-5)
decoder_optimizer = optim.Adam(decoder.parameters(), lr=0.0005, weight_decay=1e-5)
encoder_scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    encoder_optimizer, mode="min", factor=0.9, patience=2
)
decoder_scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    decoder_optimizer, mode="min", factor=0.9, patience=2
)

In [11]:
import torch
import torch.nn.functional as F
from tqdm import tqdm

seq2seq.train(
    train_loader,
    test_loader,
    encoder_optimizer,
    decoder_optimizer,
    criterion,
    device,
    encoder_scheduler,
    decoder_scheduler,
    20,
)


Epoch 1/20


OutOfMemoryError: CUDA out of memory. Tried to allocate 1.09 GiB. GPU 0 has a total capacity of 5.76 GiB of which 434.75 MiB is free. Including non-PyTorch memory, this process has 5.33 GiB memory in use. Of the allocated memory 5.20 GiB is allocated by PyTorch, and 23.09 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)